In [3]:
import os
os.environ['PATH'] = '/scratch/smp/uqsmac12/.conda/env/lit_torch_gp/bin:' + os.environ['PATH']

In [1]:
!which python

/scratch/smp/uqsmac12/.conda/env/stimage/bin/python


In [ ]:
'Q2051'

In [7]:
import os
from anndata import read_h5ad
import numpy as np

In [18]:
import pandas as pd

In [2]:
import os
import torch

from torch.utils.data import DataLoader, random_split
from torch.nn import functional as F

from torch import optim, nn, utils, Tensor
from torchvision.datasets import MNIST
from torchvision import datasets, transforms
# from torchvision.transforms import ToTensor
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

/scratch/smp/uqsmac12/.conda/env/lit_torch_gp/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# update the location where models will be saved to
if torch.hub.get_dir() == '/clusterdata/uqsmac12/.cache/torch/hub':
    torch.hub.set_dir('/scratch/smp/uqsmac12/.cache/torch/hub')

In [17]:
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
# setup new conda environment

# load data

# implement somthing similar to STimage1

# test likelihood assumption with log [CPT] data transform
# test likelihood assumption without transform

# load data

In [3]:
torch.cuda.is_available()

True

In [3]:
DIR_DATA = '/scratch/smp/uqsmac12/dropin_data'
DIR_CHECKPOINTS = '/scratch/smp/uqsmac12/dropin_data/checkpoints'
DIR_WANDB = '/scratch/smp/uqsmac12/dropin_data'

In [11]:
DIR_TILES = '/scratch/smp/uqsmac12/dataset_breast_cancer_9visium'
DIR_ANNDATA_PROCESSED = '/scratch/smp/uqsmac12/dataset_breast_cancer_9visium'
file_processed_alex_data = 'all_adata.h5ad'
# DIR_PROCESSED_DATA = '/afm03/Q2/Q2051/STimage_project/STimage_dataset/PROCESSED/dataset_breast_cancer_9visium'

In [12]:
adata_all = read_h5ad(os.path.join(DIR_ANNDATA_PROCESSED, file_processed_alex_data))

In [13]:
# update metadata for annadata such that it maps to the correct location
adata_all.obs["tile_path"] = adata_all.obs.tile_path.map(
    lambda x: x.replace("/clusterdata/uqxtan9/Xiao/breast_cancer_9visium",
                        DIR_TILES))

In [14]:
gene_list = ["COX6C","TTLL12", "PABPC1", "GNAS", "HSP90AB1", "TFF3", "ATP1A1", "B2M", "FASN", "SPARC", "CD74", "CD63", "CD24", "CD81"]

confirm normalisation of data

In [24]:
np.exp(adata_all[0:10,'COX6C'].to_df().values.reshape(-1))

array([ 8.      , 12.      , 15.000002, 17.      , 12.      , 10.000001,
       15.000002,  7.      ,  8.      ,  6.      ], dtype=float32)

In [25]:
class DataGenerator(torch.utils.data.Dataset):
    """
    data generator for multiple branches gene prediction model
    """

    def __init__(self, adata, dim=(299, 299), n_channels=3, genes=None, aug=False, tile_path="tile_path"):
        'Initialization'
        self.dim = dim
        self.adata = adata
        self.n_channels = n_channels
        self.genes = genes
        self.num_genes = len(genes)
        self.aug = aug
        self.tile_path = tile_path
        self.indexes = np.arange(self.adata.n_obs)
        
    def __len__(self):
        'Denotes the number of samples'
        return int(self.adata.n_obs)

    def __getitem__(self, index):
        'Generates one sample of data'
        # Find obs name
        obs_temp = self.adata.obs_names[index]

        # Generate data
        X_img, y = self._load_data(obs_temp)

        return torch.Tensor(X_img), torch.Tensor(y)

    def _load_data(self, obs):
        img_path = self.adata.obs.loc[obs, 'tile_path']
        X_img = Image.open(img_path).convert('RGB')
        X_img = transforms.Resize(self.dim)(X_img)
        X_img = np.array(X_img).astype('uint8')
        #         X_img = np.expand_dims(X_img, axis=0)
        #         n_rotate = np.random.randint(0, 4)
        #         X_img = np.rot90(X_img, k=n_rotate, axes=(1, 2))
        if self.aug:
            X_img = seq_aug(image=X_img)
        y = self._load_label(obs)
        return X_img, y

    def _load_label(self, obs):
        batch_adata = self.adata[obs, self.genes].copy()

        return tuple([batch_adata.to_df()[i].values for i in self.genes])

    def get_classes(self):
        return self.adata.to_df().loc[:, self.genes]

# Pytorch module

In [31]:
os.environ['TORCH_HOME']

KeyError: 'TORCH_HOME'

# lit module

In [26]:
import torchvision.models

'/clusterdata/uqsmac12/.cache/torch/hub'

In [35]:
torch.hub.set_dir('/scratch/smp/uqsmac12/.cache/torch/hub')

In [36]:
torch.hub.get_dir()

'/scratch/smp/uqsmac12/.cache/torch/hub'

In [37]:
backbone = torchvision.models.resnet18(weights="DEFAULT",)

TypeError: 'generator' object is not subscriptable

In [ ]:
class ImagenetTransferLearning(pl.LightningModule):
    def __init__(self):
        super().__init__()
        
        backbone = torchvision.models.resnet18(weights="DEFAULT")
        num_filters = backbone.fc.in_features
        layers = list(backbone.children())[:-1]
        self.feature_extractor = nn.Sequential(*layers)
        
        # use the pretrained model to classify cifar10
        num_target_classes = 10
        self.classifier = nn.Linear(num_filters, num_target_classes)
        
    def forward(self, x):
        self.feature_extractor.eval()
        with torch.no_grad():
            representations = self.feature_extractor(x).flatten(1)
        x = self.classifier(representations)

In [ ]:
# finetune
model = ImagenetTransferLearning()
trainer = Trainer()
trainer.fit(model)

In [ ]:
# # use to predict data of interest
# model = ImagenetTransferLearning.load_from_checkpoint(PATH)
# model.freeze()

# x = some_images_from_cifar10()
# predictions = model(x)

# logger

In [ ]:
wandb_logger = WandbLogger(
    project="Stimage2", 
    log_model="all",
    save_dir=DIR_WANDB,
    name='stimage1_likelihood_sensitivity' # run names
)

In [ ]:
# wandb_logger.watch(autoencoder)

# dataset

In [ ]:
# # dataset
# train_set = MNIST(root=DIR_DATA, download=True, transform=transforms.ToTensor())
# test_set = MNIST(root=DIR_DATA, download=True, train=False, transform=transforms.ToTensor())

# # use 20% of training data for validation
# train_set_size = int(len(train_set) * 0.8)
# valid_set_size = len(train_set) - train_set_size

# # split the train set into two
# seed = torch.Generator().manual_seed(42)
# train_set, valid_set = random_split(train_set, [train_set_size, valid_set_size], generator=seed)

# # dataloaders
# train_loader = DataLoader(train_set, batch_size=256, num_workers=8)
# valid_loader = DataLoader(valid_set, batch_size=256, num_workers=8)
# test_loader = DataLoader(test_set, batch_size=256, num_workers=8)

# trainer


In [ ]:
# train model
early_stopping_callback = EarlyStopping(monitor="val_loss", mode="min", patience=3,)
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    dirpath=None, monitor="val_loss", save_top_k=1, every_n_epochs=None
)

trainer = pl.Trainer(
    default_root_dir=DIR_CHECKPOINTS, 
    enable_checkpointing=True, 
    accelerator='gpu', 
    logger=wandb_logger,
    callbacks=[early_stopping_callback],
    
)

trainer.fit(model=autoencoder, train_dataloaders=train_loader, val_dataloaders=valid_loader)

# SUPERSEDED